This an implementation of the paper proposed by Niu et al. in 2021, entitled "*An improved learnable evolution model for solving multi-objective vehicle routing problem with stochastic demand*".

Briefly speaking, the paper uses evolution algorithm along with decision tree classifier to solve the vehicle routing problem in which the clients have stochastic demands. Moreover, several objectives have been considered in this algorithm including the distance, total time spent, and the number of sub-routes which is also referred to as the number of vehicles.

The implemented algorithm is then tested on Solomon dataset and the results are reported.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re
from scipy.stats import pareto

# IMOLEM

The dataset, number of specified problem instance and the number of clients on which the problem is being run can be selected as int the following cell.

In [5]:
dataset_num = 7
dataset = "C"
client_number = 25

### Data Preparation

The path should be modified according to the file path.

In [6]:
if dataset_num <=9:
  if dataset == "RC":
    data_path = "/Datasets/Solomon/RC1/rc10"+str(dataset_num)+".txt"
  elif dataset == "C":
    data_path = "/Datasets/Solomon/C1/c10"+str(dataset_num)+".txt"
  elif dataset == "R":
    data_path = "/Datasets/Solomon/R1/r10"+str(dataset_num)+".txt"
else:
    data_path = "/Datasets/Solomon/R1/r1"+str(dataset_num)+".txt"

data_dataframe = pd.read_csv(data_path, delim_whitespace=True)

In [7]:
coordinates = dict()
time_windows = dict()
demands = dict()
service_times = dict()

coordinates[0] = [data_dataframe["XCOORD."][0], list(data_dataframe["YCOORD."])[0]]
time_windows[0] = [data_dataframe["READY_TIME"][0], list(data_dataframe["DUE_DATE"])[0]]
demands[0] = list(data_dataframe["DEMAND"])[0]
service_times[0] = list(data_dataframe["SERVICE_TIME"])[0]

for item in list(dict(data_dataframe["XCOORD."]).keys())[1:client_number+1]:
  coordinates[item] = [data_dataframe["XCOORD."][item], list(data_dataframe["YCOORD."])[item]]
  time_windows[item] = [data_dataframe["READY_TIME"][item], list(data_dataframe["DUE_DATE"])[item]]
  demands[item] = list(data_dataframe["DEMAND"])[item]
  service_times[item] = list(data_dataframe["SERVICE_TIME"])[item]

In [8]:
print(coordinates)
print(time_windows)
print(demands)
print(service_times)

{0: [40.0, 50.0], 1: [45.0, 68.0], 2: [45.0, 70.0], 3: [42.0, 66.0], 4: [42.0, 68.0], 5: [42.0, 65.0], 6: [40.0, 69.0], 7: [40.0, 66.0], 8: [38.0, 68.0], 9: [38.0, 70.0], 10: [35.0, 66.0], 11: [35.0, 69.0], 12: [25.0, 85.0], 13: [22.0, 75.0], 14: [22.0, 85.0], 15: [20.0, 80.0], 16: [20.0, 85.0], 17: [18.0, 75.0], 18: [15.0, 75.0], 19: [15.0, 80.0], 20: [30.0, 50.0], 21: [30.0, 52.0], 22: [28.0, 52.0], 23: [28.0, 55.0], 24: [25.0, 50.0], 25: [25.0, 52.0]}
{0: [0.0, 1236.0], 1: [850.0, 1030.0], 2: [758.0, 938.0], 3: [16.0, 196.0], 4: [665.0, 845.0], 5: [15.0, 195.0], 6: [572.0, 752.0], 7: [108.0, 288.0], 8: [200.0, 380.0], 9: [480.0, 660.0], 10: [294.0, 474.0], 11: [387.0, 567.0], 12: [597.0, 777.0], 13: [30.0, 210.0], 14: [504.0, 684.0], 15: [317.0, 497.0], 16: [412.0, 592.0], 17: [34.0, 214.0], 18: [127.0, 307.0], 19: [222.0, 402.0], 20: [10.0, 190.0], 21: [850.0, 1030.0], 22: [758.0, 938.0], 23: [665.0, 845.0], 24: [15.0, 195.0], 25: [107.0, 287.0]}
{0: 0.0, 1: 10.0, 2: 30.0, 3: 10.0,

## Functions

In [9]:
from scipy.spatial import distance
from sklearn import tree
import numpy as np
import random

In [10]:
def pareto_frontier_multi(myArray):
    # Sort on first dimension
    myArray = myArray[myArray[:,0].argsort()]
    # Add first row to pareto_frontier
    pareto_frontier = myArray[0:1,:]
    # Test next row against the last row in pareto_frontier
    for row in myArray[1:,:]:
        if sum([row[x] >= pareto_frontier[-1][x]
                for x in range(len(row))]) == len(row):
            # If it is better on all features add the row to pareto_frontier
            pareto_frontier = np.concatenate((pareto_frontier, [row]))
    return pareto_frontier

In [11]:
def compute_distances(clients):
    ## Computes the total distance matrix between each pair of clients
    distances = []
    for from_client in clients:
        row = []
        for to_client in clients:
          row.append(distance.euclidean(from_client, to_client))
        distances.append(row)
    return np.array(distances)

def sub_routes_to_chromosome(routes):
  ## Converts a route including several sub-routes into a chromosone
  individual = []
  for item in routes:
    for i in item:
      individual.append(i)
  return individual

def subroute_distance(sequence, distances):
    ## Calculates the total distance within a sub-route, given the sequence and the distance matrix
    dist = 0
    key = 1
    seq_dict = dict()
    seq_dict[0] = 0
    for s in sequence:
      seq_dict[key] = s
      key +=1
    sorted_route = dict(sorted(seq_dict.items(), key=lambda x:x[1], reverse=True))
    # print(seq_dict)
    # print(sorted_route)
    Keys = list(sorted_route.keys())
    Values = list(sorted_route.values())

    # print("Keys => ",Keys)
    # print("Values => ",Values)

    rt = [0]+sequence+[0]
    for item in range(len(rt)-1):
      # print(rt[item], rt[item+1], Values.index(rt[item]), Values.index(rt[item+1]))
      key1 = Keys[Values.index(rt[item])]
      key2 = Keys[Values.index(rt[item+1])]
      # print("Keys => ", key1, key2)
      dist += distances[(key1, key2)]
      # print(rt[item], rt[item+1], "Keys => ", key1, key2, dist)

    return dist

def route_distance(sequence, distances):
    ## Calculates the total distance of a route comprised of several sub-routes
    dist = 0
    routes = []
    key = 1
    seq_dict = dict()
    seq_dict[0] = 0
    for s in sequence:
      seq_dict[key] = s
      key +=1
    sorted_route = dict(sorted(seq_dict.items(), key=lambda x:x[1], reverse=True))
    # print(seq_dict)
    # print(sorted_route)
    Keys = list(sorted_route.keys())
    Values = list(sorted_route.values())

    # print("Keys => ",Keys)
    # print("Values => ",Values)

    i1 = 0
    for i in range(len(Keys)-1):
      if np.abs(Values[i+1]-Values[i])>1:
        # print("Found one sub-route", i, Values[i1:i+1])
        r = Values[i1:i+1]
        routes.append(r)
        i1 = i+1

    # print(routes)

    for rt in routes:
      rt = [0]+rt+[0]
      for item in range(len(rt)-1):
        # print(rt[item], rt[item+1], Values.index(rt[item]), Values.index(rt[item+1]))
        key1 = Keys[Values.index(rt[item])]
        key2 = Keys[Values.index(rt[item+1])]
        dist += distances[key1, key2]
        # print(rt[item], rt[item+1], "Keys => ", key1, key2, dist)

    return dist

def sub_route_identifier(sequence):
    ## Identifies the sub-routes within a sequence and
    ## Converts a sequence into a list of sub-routes
    routes = []
    key = 1
    seq_dict = dict()
    for s in sequence:
      seq_dict[key] = s
      key +=1

    Keys = list(seq_dict.keys())
    Values = list(seq_dict.values())
    i1 = 0
    for i in range(len(Keys)-1):
      if np.abs(Values[i+1]-Values[i])>1:
        r = Values[i1:i+1]
        routes.append(r)
        i1 = i+1
    routes.append(Values[i1:])
    return routes

def get_swap_dict(d):
    ## Swapps the value-key association of given dictionary
    return {v: k for k, v in d.items()}

In [12]:
distance_matrix = compute_distances(list(coordinates.values()))
distance_matrix.shape

(26, 26)

## Classes

### Chromosome

In [14]:
class chromosome_class():
  def __init__(self, length, max_priority, initialize=False):
    self.length = length
    self.max_priority = max_priority
    self.chromosome = []

    if initialize:
      self.initialize()

  def initialize(self):
    ## According to the random policy mentioned in the paper,
    ## the priorities chromosomes will be initialized randomly.
    ## The bubbles in a chromosome are randomly created using a rand variable.
    max_prior = self.max_priority
    for i in range(1,self.length+1):
      rand_ = random.random()
      if rand_ <0.4:
        max_prior -= 1
        self.chromosome.append(max_prior)
      else:
        self.chromosome.append(max_prior)
      max_prior -= 1

    ## Here, to randomize the sequence of clients within a route,
    ## we randomly shuffle the sub-routes to create a random individual.
    routes = sub_route_identifier(self.chromosome)
    random.shuffle(routes)
    self.chromosome = sub_routes_to_chromosome(routes)

### Population

In [15]:
from numpy.random import randint
class population():
  def __init__(self, generation, length, vehicle_range, demands, vehicle_capacity, time_windows, distances, initialize=False):
    self.size = generation
    self.length = length
    self.vehicle_range = vehicle_range
    self.client_demands = demands
    self.vehicle_capacity = vehicle_capacity
    self.client_time_windows = time_windows
    self.client_number = len(self.client_demands)
    self.N_r = self.client_number ## initially the number of sub-routes is considered the number of clients as we haven't created any chromosome
    self.distances = distances

    self.vehicle_number = randint(1,self.vehicle_range)
    print("vehicle number => ",self.vehicle_number)
    self.max_priority = self.client_number + self.N_r-1
    print("max priority => ", self.max_priority)

    self.population = []

    if initialize:
      self.initialize()

  def demand_check(self, sequence): ## The total demands of the clients in a sub-route should not be greater than the vehicle-capacity
    total_demand = 0
    for i in range(len(sequence)):
      total_demand += self.client_demands[i]
    return total_demand <= self.vehicle_capacity

  def initialize(self):
    max_nv = self.vehicle_number
    i = 0
    while i <= self.size:
      ch = chromosome_class(self.length, self.max_priority, initialize=True)
      routes = sub_route_identifier(ch.chromosome)
      demand_flag = True
      for subroute in routes:
        if not self.demand_check(subroute):
          flag = False
          break
      time_flag = True
      for subroute in routes: ## we assume the time is equal to the distance
        dist = subroute_distance(subroute, self.distances)
        if dist >= 0.8*self.client_time_windows[0][1]:
          time_flag = False
          break

      if demand_flag and time_flag and len(routes)<= max_nv: ## the chromosome is accepted
        if i == 0:
          max_nv = len(routes)
        self.population.append(ch.chromosome)
        i += 1

  def create(self, tree, n_generation): ## creating new generation of individuals according to the positive rules of the trained tree model
    pop_H = []
    self.n_generation = n_generation
    print("H generated population => ",int(self.n_generation/2))
    ## Half of the generation value should be constructed using the trained tree model
    for i in range(int(self.n_generation/2)):
      ch = chromosome_class(self.length, self.max_priority, initialize=True)
      prd = tree.predict([ch.chromosome])[0]
      rep = 0 ## rep prevents an unending loop
      while prd==0 and rep<100000:
        rep += 1
        ch = chromosome_class(self.length, self.max_priority, initialize=True)
        prd = tree.predict([ch.chromosome])[0]
      pop_H.append(ch.chromosome)
      print(i, prd, ch.chromosome)

    ## The other hald of the generation should be created using random initialization
    pop_Random = []
    for i in range(int(self.n_generation/2), self.n_generation):
      ch = chromosome_class(self.length, self.max_priority, initialize=True)
      pop_Random.append(ch.chromosome)
    return pop_H+pop_Random

  def partial_swapping(self, individual):
    routes = sub_route_identifier(individual)
    # print("Routes before the partial swap => ", routes)
    if len(routes)>=2:## Choose two subroutes to partial swap their randomly selected segments
      index1 = random.randint(0, len(routes)-1)
      index2 = random.randint(0, len(routes)-1)
      while index1 == index2:
        index2 = random.randint(0, len(routes)-1)

      min_index = min(index1, index2)
      max_index = max(index1, index2)
      sub_routes = [routes[min_index], routes[max_index]]
      # print("Sub routes =>", sub_routes)

      segments= []
      for route in sub_routes:
        if len(route)>=2:
          idx1 = random.randint(0, len(route)-1)
          idx2 = random.randint(0, len(route)-1)
          while idx1 == idx2:
            idx1 = random.randint(0, len(routes)-1)
          min_index = min(idx1, idx2)
          max_index = max(idx1, idx2)
          segments.append([min_index, max_index, route[min_index: max_index]])
        else:
          segments.append([0, 1, route])

      # print("Segments => ", segments)
      value_0 = segments[0][2][:]
      value_1 = segments[1][2][:]
      # print("values => ", value_0, value_1)
      sub_routes[0][segments[0][0]:segments[0][1]] = value_1
      sub_routes[1][segments[1][0]:segments[1][1]] = value_0
      # print("Swapped routes => ", sub_routes[0], sub_routes[1])

      routes[index1] = sub_routes[0]
      routes[index2] = sub_routes[1]
      # print("Routes after partial swap = >", routes)
    else:
        return individual
    return sub_routes_to_chromosome(routes)

  def priority_fix(self, routes):
    # print("Inside Priority Fix => ", routes)

    individual_dict = dict()
    individual = []
    for subroute in routes:
      individual += subroute
    individual_dict = dict(enumerate(individual))
    index_dict = dict(enumerate(sorted(list(individual_dict.values()), reverse=True)))
    index_dict = get_swap_dict(index_dict)
    # print("individual_dict => ", individual_dict)
    # print("index_dict => ", index_dict)

    paths = []
    for route in routes:
      path = []
      for i in route:
        path.append(index_dict[i])
      paths.append(path)
    # print(paths)

    max_pr = len(individual_dict.keys())+len(routes)-1
    # print(len(individual_dict.keys()), len(routes), max_pr)
    for path in paths:
      d = dict(sorted(dict(enumerate(path)).items(), key=lambda x:x[1])).values()
      d = dict(enumerate(d))
      for key in d:
        path[path.index(d[key])] = max_pr
        max_pr -= 1
      max_pr -= 1
    # print("Fixed Routes => ", paths)
    return paths

  def merging_sub_routes(self, individual, distances):
    routes = sub_route_identifier(individual)
    # print("inside merging subroutes", routes)
    if len(routes)>=2:
      subroute_dists = dict()
      for route_index in range(len(routes)):
        subroute_dists[route_index] = subroute_distance(routes[route_index], distances)
      # print(subroute_dists)

      sorted_dists = sorted(subroute_dists.items(), key=lambda x:x[1])
      # print(sorted_dists)

      min_index = min(sorted_dists[0][0], sorted_dists[1][0])
      max_index = max(sorted_dists[0][0], sorted_dists[1][0])

      merged_route = routes[min_index]+routes[max_index]
      # print("merged route => ", merged_route)
      routes[min_index] = merged_route
      routes.remove(routes[max_index])
      routes = self.priority_fix(routes)

    return sub_routes_to_chromosome(routes)

  def split_routes(self, individual, distances):
    routes = sub_route_identifier(individual)
    # print("inside splitting subroutes", routes)

    subroute_dists = dict()
    for route_index in range(len(routes)):
      subroute_dists[route_index] = subroute_distance(routes[route_index], distances)
    # print(subroute_dists)

    sorted_dists = sorted(subroute_dists.items(), key=lambda x:x[1], reverse=True)
    # print(sorted_dists)
    route_index = sorted_dists[0][0]
    longest_route = routes[route_index]
    # print("Longest sub-route =>", longest_route)

    rand_position = random.randint(1,len(longest_route)-1)
    split_1 = longest_route[0:rand_position]
    split_2 = longest_route[rand_position:]
    routes.insert(route_index, split_1)
    routes.insert(route_index+1, split_2)
    routes.remove(longest_route)
    routes = self.priority_fix(routes)
    # print("Rand position and split sections => ", rand_position, split_1, split_2)
    # print("routes after splitting the longest route => ", routes)
    return sub_routes_to_chromosome(routes)

  def random_suffle(self, individual):
    routes = sub_route_identifier(individual)
    # print("inside random shuffle", routes)

    rand_indx = random.randint(0, len(routes)-1)
    random.shuffle(routes[rand_indx])
    # print("After shuffling => ", rand_indx, routes)
    return sub_routes_to_chromosome(routes)

  def heuristic_operations(self, individual, distances, swap_rate, merge_rate, shuffle_rate):
    swap_rand = random.random()
    merge_rand = random.random()
    shuffle_rand = random.random()

    resulted_individual = individual

    if swap_rand < swap_rate:
      resulted_individual = self.partial_swapping(resulted_individual)
      # print("Individual after partial swapping => ", resulted_individual)
      # print("============= Partial Swapping Done ================")
    elif merge_rand < merge_rate:
      resulted_individual = self.merging_sub_routes(resulted_individual, distances)
      # print("Individual after merging => ", resulted_individual)
      # print("============== Merging Done ===============")
    else:
      resulted_individual = self.split_routes(resulted_individual, distances)
      # print("Individual after Splitting => ", resulted_individual)
      # print("============== Splitting the longest Done ====================")

    if shuffle_rand < shuffle_rate:
      resulted_individual = self.random_suffle(resulted_individual)
      # print("Individual after Shuffling => ", resulted_individual)
      # print("============== Random Shuffling Done ====================")

    # print(resulted_individual)
    return resulted_individual

In [16]:
def driver_renumeration(route, distances, service_time, B, m1, m2, Q_r):
  distance = route_distance(route, distances)

  total_service_time = 0
  for r in range(len(route)):
    total_service_time += service_time[r]
  time_duration = distance+total_service_time

  additional_time = 0
  for item in Q_r:
      additional_time += service_time[r]
  total_service_time += additional_time
  W = total_service_time

  M = 0
  if time_duration <= B:
    M = (W/B)*time_duration*m1
  else:
    M = W*m1+((W/B)*time_duration-W)*m2

  return M

In [17]:
import copy
def RSM(individual, distances_matrix, client_demands, client_time_window, capacity, service_times, B, m1, m2):
  distances = []
  n_subroutes = []
  M_values = []

  v_cap = capacity
  routes = sub_route_identifier(individual)
  route_keys = dict(enumerate(individual))
  # print(routes)
  # print(route_keys)
  n = len(client_demands)
  # n = 5
  additional_cost = []

  max_pr = max(individual)
  sorted_values = sorted(list(route_keys.values()), reverse=True)
  # print(sorted_values)
  for i in range(n):
    Q_r = []
    v_cap = capacity
    left_demands = copy.deepcopy(client_demands)
    left_demands.pop(0)
    # print("before assigning the demanads",left_demands)

    for key in left_demands:
      mean_value = left_demands[key]
      sigma = np.random.uniform(0,0.33)*mean_value
      left_demands[key] = np.random.normal(mean_value, sigma)
      # rand = random.random()
      # if rand>0.5:
      #   left_demands[key] = mean_value + np.random.normal(mean_value, sigma)
      # else:
      #   left_demands[key] = mean_value - np.random.normal(mean_value, sigma)
    # print("after assigning the demands", left_demands)

    left_demands_keys = list(left_demands.keys())
    j = 0
    while len(left_demands_keys):
      indx = sorted_values[j]
      client_index = list(route_keys.keys())[list(route_keys.values()).index(indx)]+1
      # print(f"index={indx}" , f"client index={client_index}", left_demands[client_index], v_cap)
      if v_cap >= left_demands[client_index]:
        v_cap-= left_demands[client_index]
        left_demands.pop(client_index)
        j += 1
      else:
        # print("Not", left_demands[client_index], v_cap)
        ## serve the client partially
        left_demands[client_index] -= v_cap
        ## return to depot to refill the capacity
        v_cap = capacity
        Q_r.append(client_index)

      left_demands_keys = list(left_demands.keys())
      # print("===============", left_demands)
    additional_cost.append(Q_r)

    for item in Q_r:
      added_subroute = [item, 0, item]
      # print(added_subroute)
      added_distance = 0
      for item in range(len(added_subroute)-1):
        added_distance += distances_matrix[added_subroute[item], added_subroute[item+1]]
      # print("Added distance", added_distance)

    dist = route_distance(individual, distances_matrix) + added_distance
    distances.append(dist)
    n_subroutes.append(len(routes))
    obj_M = driver_renumeration(individual, distances_matrix, service_times, B, m1, m2, Q_r)
    M_values.append(obj_M)

  # print(additional_cost)
  # print(distances)
  # print(M_values)
  return np.mean(distances), np.mean(M_values), np.mean(n_subroutes)

# Main

In [18]:
iteration = 50
population_size = 100
n_generation = 200
length = len(coordinates)-1
vehicle_capacity = 100
H_group_rate = 0.3
L_group_rate = 0.3

search_rate = 0.4
swapping_rate = 0.5
merging_rate = 0.5
shuffling_rate =  0.3

B= 0.8*time_windows[0][1]
m1 = 10
m2 = 20

H_group = []
L_group = []

Population = population(population_size, length, client_number, demands, vehicle_capacity, time_windows, distance_matrix, initialize=True)
P = Population.population
print(f"P => {len(P)}",P)

Q = []
clf = tree.DecisionTreeClassifier(criterion='entropy', random_state=0)
for iter in range(iteration):
  print(f"\n ========= Iteration {iter} =========== \n")
  obj_dict_Q = dict()
  for individual in Q:
    obj_distance,obj_M, obj_subroutes = RSM(individual, distance_matrix, demands, time_windows, vehicle_capacity,service_times, B, m1, m2)

    objective_values = [obj_distance, obj_M, obj_subroutes]
    obj_dict_Q[Q.index(individual)] = objective_values


  Q.extend(P)
  if len(obj_dict_Q):
    obj_list = np.array(list(obj_dict_Q.values()))
    pt_Q_values = pareto_frontier_multi(obj_list)

    Q_sorted = []
    for i in pt_Q_values:
      index = list(obj_dict_Q.values()).index(list(i))
      Q_sorted.append(P[index])
    Q = Q_sorted[:population_size]
    # print("Q sorted =>", Q)

  obj_dict_P = dict()
  for individual in P:
    obj_distance, obj_M, obj_subroutes = RSM(individual, distance_matrix, demands, time_windows, vehicle_capacity,service_times, B, m1, m2)
    objective_values = [obj_distance, obj_M, obj_subroutes]

    obj_dict_P[P.index(individual)] = objective_values


  pt_P_values = pareto_frontier_multi(np.array(list(obj_dict_P.values())))
  # print("Pareto sort", len(pt_P_values), pt_P_values)
  print("Pareto sort", len(pt_P_values))
  P_sorted = []
  for i in pt_P_values:
    index = list(obj_dict_P.values()).index(list(i))
    P_sorted.append(P[index])

  P = P_sorted
  # print("P sorted => ",P)
  print(len(P), int(H_group_rate*len(P)))
  H_group = P[0: int(H_group_rate*len(P))]
  L_group = P[-int(L_group_rate*len(P)):]
  if len(H_group)==0 and len(P)>1:
    H_group = P[0:int(len(P)/2)]
    L_group = P[-int(len(P)/2):]

  if len(P)==1:
    H_group = P
    L_group = []

  X = [item for item in H_group]
  Y = [[1] for item in H_group]
  X +=[item for item in L_group]
  Y += [[0] for item in L_group]

  print("H_group", H_group)
  print("L_group", L_group)
  clf.fit(X,Y)

  print()
  indexes = ["Client "+str(index) for index in range(1,length+1)]
  text_representation = tree.export_text(clf, feature_names=indexes)
  print(text_representation)

  new_generation = Population.create(clf, n_generation)
  # print("New generation ", len(new_generation) , new_generation)
  print("New generation ", len(new_generation))
  new_generation.extend(Q[0:int(len(Q)/2)])
  # print("New generation ", len(new_generation) , new_generation)
  print("New generation ", len(new_generation))
  P = new_generation
  Population.population = P
  for individual in P[int(len(P)/2):]:
    index = P.index(individual)
    P[index] = Population.heuristic_operations(P[index], distance_matrix, swapping_rate, merging_rate, shuffling_rate)

  # print("P", len(P) , P)
  print("P", len(P))

print("Q is :")
print(Q)

Streaming output truncated to the last 5000 lines.

 ========= Iteration 8 =========== 

Pareto sort 10
10 3
H_group [[28, 51, 48, 46, 49, 47, 45, 44, 50, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 25, 27, 23], [37, 36, 20, 19, 33, 32, 31, 30, 26, 34, 28, 44, 43, 48, 46, 49, 50, 51, 47, 41, 40, 39, 24, 23, 22], [34, 33, 31, 30, 29, 27, 26, 24, 20, 21, 22, 19, 14, 15, 16, 17, 13, 11, 9, 5, 6, 7, 8, 3, 1]]
L_group [[40, 38, 37, 34, 33, 35, 31, 30, 28, 26, 25, 24, 22, 21, 19, 17, 15, 13, 12, 10, 8, 6, 4, 2, 1], [36, 25, 20, 19, 51, 12, 47, 46, 15, 13, 50, 49, 23, 22, 29, 34, 44, 43, 42, 40, 27, 17, 38, 32, 31], [21, 39, 34, 33, 25, 27, 19, 37, 36, 10, 14, 51, 29, 17, 16, 31, 49, 23, 12, 47, 46, 45, 44, 43, 41]]

|--- Client 16 <= 35.00
|   |--- Client 21 <= 7.00
|   |   |--- class: 1
|   |--- Client 21 >  7.00
|   |   |--- class: 0
|--- Client 16 >  35.00
|   |--- class: 1

H generated population =>  100
0 1 [49, 48, 47, 46, 45, 44, 43, 42, 41, 51, 34, 33, 32, 31, 19, 36, 39, 38,

In [19]:
results = []
for item in Q:
  dist, M, n = RSM(item, distance_matrix, demands, time_windows, vehicle_capacity,service_times, B, m1, m2)
  results.append([dist, M, n])

In [20]:
results

[[498.5273111309964, 108519.63867916787, 10.0]]

In [21]:
print(f"Dataset {dataset}, {dataset_num}, {client_number}")
print(np.mean(results, axis=0)[0])
print(np.mean(results, axis=0)[1])
print(np.mean(results, axis=0)[2])

Dataset C, 7, 25
498.5273111309964
108519.63867916787
10.0
